In [ ]:
# Import necessary libraries
import sys
import os
import torch
import pandas as pd
import numpy as np
from pathlib import Path

# Add src to path for our custom modules
sys.path.append(str(Path.cwd().parent / "src"))

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("GPU not available, using CPU")


In [ ]:
# Configuration for our LoRA training
config = {
    "model": {
        "model_name": "microsoft/DialoGPT-medium",  # Using a smaller model for tutorial
        "cache_dir": "../model_cache",
        "torch_dtype": "auto"
    },
    "lora": {
        "r": 16,                    # Rank of adaptation
        "lora_alpha": 32,           # LoRA scaling parameter
        "target_modules": [         # Which modules to apply LoRA to
            "c_attn",               # Attention projections
            "c_proj", 
            "c_fc"                  # Feed-forward layers
        ],
        "lora_dropout": 0.1,        # Dropout for LoRA layers
        "bias": "none",             # Whether to adapt bias parameters
        "task_type": "CAUSAL_LM"    # Type of task
    },
    "data": {
        "data_path": "../data/sample_data.json",
        "max_seq_length": 512,
        "train_on_inputs": False,   # Only train on assistant responses
        "validation_split": 0.1
    },
    "training": {
        "output_dir": "./outputs",
        "num_train_epochs": 1,      # Reduced for tutorial
        "per_device_train_batch_size": 2,
        "gradient_accumulation_steps": 4,
        "learning_rate": 3e-4,
        "warmup_steps": 50,
        "logging_steps": 10,
        "save_steps": 100,
        "eval_steps": 100,
        "bf16": True,               # Use mixed precision
        "gradient_checkpointing": True,
        "report_to": [],            # Disable wandb for tutorial
    }
}

print("Configuration loaded successfully!")


In [ ]:
# Load and setup the model with LoRA adapters
from model_setup import ModelSetup
from data_preprocessing import DataProcessor, ChatDataCollator

# Initialize model setup
print("Setting up model...")
model_setup = ModelSetup(config)

# Load model and tokenizer
model, tokenizer = model_setup.setup_model_and_tokenizer()

# Print model information
def print_model_info(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n{'='*50}")
    print(f"MODEL INFORMATION")
    print(f"{'='*50}")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Percentage trainable: {100 * trainable_params / total_params:.2f}%")
    print(f"{'='*50}\n")

print_model_info(model)


In [ ]:
# Setup data processor
data_processor = DataProcessor(
    tokenizer=tokenizer,
    max_length=config["data"]["max_seq_length"],
    train_on_inputs=config["data"]["train_on_inputs"]
)

# Load and prepare datasets
print("Loading datasets...")
datasets = data_processor.prepare_datasets(
    dataset_name="local",
    data_path=config["data"]["data_path"],
    validation_split=config["data"]["validation_split"],
    num_proc=4
)

print(f"Training examples: {len(datasets['train'])}")
print(f"Validation examples: {len(datasets['validation'])}")

# Create data collator
data_collator = ChatDataCollator(
    tokenizer=tokenizer,
    max_length=config["data"]["max_seq_length"],
    train_on_inputs=config["data"]["train_on_inputs"]
)

# Let's look at a sample
sample_example = datasets["train"][0]
print("\nSample tokenized example:")
print(f"Input IDs length: {len(sample_example['input_ids'])}")
print(f"Labels length: {len(sample_example['labels'])}")

# Decode to see the actual text
decoded_text = tokenizer.decode(sample_example['input_ids'])
print(f"\nDecoded text: {decoded_text[:200]}...")


In [ ]:
# Setup trainer
from trainer import LoRATrainer

print("Setting up trainer...")
trainer = LoRATrainer(
    model=model,
    tokenizer=tokenizer,
    config=config
)

# Start training
print("Starting training...")
training_results = trainer.train(
    train_dataset=datasets["train"].select(range(50)),  # Use subset for demo
    eval_dataset=datasets["validation"].select(range(20)),
    data_collator=data_collator
)

print("Training completed!")
print(f"Final training loss: {training_results.get('train_loss', 'N/A')}")
print(f"Training time: {training_results.get('train_runtime', 'N/A'):.2f} seconds")


In [ ]:
# Setup inference
from inference import LoRAInference

inference = LoRAInference(model, tokenizer, config)

# Test prompts
test_prompts = [
    "Explain machine learning in simple terms.",
    "What are the benefits of renewable energy?",
    "How does photosynthesis work?",
    "Write a short story about a robot learning to paint."
]

print("🤖 Testing the trained model:\n" + "="*60)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n[{i}] Prompt: {prompt}")
    print("-" * 40)
    
    response = inference.chat(prompt)
    print(f"Response: {response}")
    print("-" * 40)
